In [1]:
import pickle
from collections import Counter
from pathlib import Path
from tqdm.notebook import tqdm
from kiwipiepy import Kiwi

PROJECT_ROOT = Path('..').resolve()
DATA_DIR = PROJECT_ROOT / 'data'

print("전체 데이터 로드")
with open(DATA_DIR / 'corpus.pkl', 'rb') as f:
    full_corpus = pickle.load(f)

print(f"분석 대상: {len(full_corpus):,}개 문서 (전체 데이터)")

kiwi = Kiwi(num_workers=-1)
token_counter = Counter()

print("전체 문서 토큰화 및 빈도 분석")

for doc in tqdm(full_corpus, desc="Full Scanning"):
    try:
        tokens = kiwi.tokenize(doc['text'])
        useful_tags = ['NNG', 'NNP', 'VV', 'VA', 'MAG']

        for t in tokens:
            if t.tag in useful_tags and len(t.form) > 1:
                token_counter[t.form] += 1
    except:
        continue

전체 데이터 로드
분석 대상: 50,222개 문서 (전체 데이터)
전체 문서 토큰화 및 빈도 분석


Full Scanning:   0%|          | 0/50222 [00:00<?, ?it/s]

여기부터 해야 됨

In [2]:
# 상위 100개
for i, (word, count) in enumerate(token_counter.most_common(100), 1):
    print(f"{i:<3}위: {word:<10} ({count:,}회)")

1  위: 나오         (276,658회)
2  위: 경우         (275,663회)
3  위: 보이         (236,684회)
4  위: 이후         (207,159회)
5  위: 사람         (199,484회)
6  위: 정도         (196,743회)
7  위: 자신         (183,742회)
8  위: 사용         (176,633회)
9  위: 가능         (170,889회)
10 위: 대하         (161,748회)
11 위: 게임         (160,336회)
12 위: 위하         (157,905회)
13 위: 사실         (143,225회)
14 위: 공격         (140,253회)
15 위: 만들         (137,953회)
16 위: 등장         (132,425회)
17 위: 이상         (114,953회)
18 위: 경기         (114,795회)
19 위: 문제         (114,654회)
20 위: 모습         (114,001회)
21 위: 시작         (113,786회)
22 위: 시즌         (109,314회)
23 위: 가지         (108,055회)
24 위: 생각         (106,710회)
25 위: 따르         (106,601회)
26 위: 당시         (104,326회)
27 위: 함께         (101,847회)
28 위: 상대         (99,809회)
29 위: 시간         (97,972회)
30 위: 다시         (97,838회)
31 위: 상황         (97,622회)
32 위: 이름         (95,833회)
33 위: 가장         (95,275회)
34 위: 또한         (94,721회)
35 위: 한국         (94,628회)
36 위: 모두         (92,462회)
3

In [3]:
import pickle
import math
import os
from pathlib import Path
from kiwipiepy import Kiwi
from tqdm.notebook import tqdm

PROJECT_ROOT = Path('..').resolve()
DATA_DIR = PROJECT_ROOT / 'data'

print("데이터 로드 중")
with open(DATA_DIR / 'corpus.pkl', 'rb') as f:
    corpus = pickle.load(f)

print(f"분석 대상 문서 수(N): {len(corpus):,}개 (전체 데이터)")

candidate_stopwords = [
    # 5만 개 데이터 빈도 상위 100위 (검증 대상)
    '나오', '경우', '보이', '이후', '사람', '정도', '자신', '사용', '가능', '대하',
    '게임', '위하', '사실', '공격', '만들', '등장', '이상', '경기', '문제', '모습',
    '시작', '시즌', '가지', '생각', '따르', '당시', '함께', '상대', '시간', '다시',
    '상황', '이름', '가장', '또한', '한국', '모두', '결국', '다만', '많이', '일본',
    '캐릭터', '선수', '카드', '상태', '물론', '시리즈', '능력', '현재', '매우', '추가',
    '존재', '지역', '다르', '자체', '기록', '달리', '통하', '이유', '무기', '특히',
    '세계', '해당', '당하', '스킬', '역시', '거의', '인하', '레벨', '필요', '모르',
    '부분', '들어가', '다음', '효과', '처음', '바로', '없이', '미국', '사건', '부르',
    '사이', '같이', '본인', '국가', '관련', '진행', '리그', '전투', '참고', '방송',
    '그냥', '수준', '대표', '이야기', '이용', '마지막', '대부분', '일반', '떨어지', '주인공',

    # 기존 불용어 (기능어)
    '있다', '없다', '하다', '되다', '않다', '그렇다', '아니',
    '때문', '이것', '저것', '그것'
]

# 비교용 키워드
comparison_keywords = ['대학교', '인공지능', '자연']
candidate_stopwords.extend(comparison_keywords)

kiwi = Kiwi(num_workers=-1)
check_list = list(set(candidate_stopwords))
doc_freq = {word: 0 for word in check_list}

print(f"전체 문서 스캔 중 (총 {len(check_list)}개 단어 체크)")

for doc in tqdm(corpus, desc="Full Scanning"):
    try:
        tokens = kiwi.tokenize(doc['text'])
        unique_tokens = set(t.form for t in tokens)

        for word in check_list:
            if word in unique_tokens:
                doc_freq[word] += 1
    except:
        continue

데이터 로드 중
분석 대상 문서 수(N): 50,222개 (전체 데이터)
전체 문서 스캔 중 (총 114개 단어 체크)


Full Scanning:   0%|          | 0/50222 [00:00<?, ?it/s]

In [4]:
print("\n" + "="*70)
print(f"{'순위':<4} | {'단어':<10} | {'DF (등장 문서 수)':<15} | {'IDF 점수':<10} | {'판정 가이드'}")
print("="*70)

results = []
N = len(corpus)

for word, df in doc_freq.items():
    if df == 0:
        idf = 0.0
    else:
        idf = math.log(N / df)
    results.append((word, df, idf))

results.sort(key=lambda x: x[2])

# 판정 기준 (Threshold)
IDF_CUTOFF = 1.5

for i, (word, df, idf) in enumerate(results, 1):
    if word in comparison_keywords:
        judgement = "기준 키워드"
    elif idf < IDF_CUTOFF:
        if word in ['게임', '한국', '미국', '일본', '캐릭터', '선수', '리그', '시즌', '공격', '스킬']:
            judgement = "보존(도메인 키워드)"
        else:
            judgement = "삭제"
    else:
        judgement = "보존"

    print(f"{i:<4} | {word:<10} | {df:<15,} | {idf:.4f}     | {judgement}")

print("="*70)


순위   | 단어         | DF (등장 문서 수)    | IDF 점수     | 판정 가이드
1    | 있다         | 0               | 0.0000     | 삭제
2    | 없다         | 0               | 0.0000     | 삭제
3    | 그렇다        | 0               | 0.0000     | 삭제
4    | 아니         | 38,208          | 0.2734     | 삭제
5    | 때문         | 38,093          | 0.2764     | 삭제
6    | 나오         | 36,965          | 0.3065     | 삭제
7    | 보이         | 34,939          | 0.3628     | 삭제
8    | 이후         | 34,897          | 0.3641     | 삭제
9    | 정도         | 32,801          | 0.4260     | 삭제
10   | 위하         | 31,514          | 0.4660     | 삭제
11   | 가지         | 30,853          | 0.4872     | 삭제
12   | 사람         | 30,503          | 0.4986     | 삭제
13   | 만들         | 30,430          | 0.5010     | 삭제
14   | 경우         | 30,191          | 0.5089     | 삭제
15   | 사실         | 30,135          | 0.5108     | 삭제
16   | 시작         | 28,113          | 0.5802     | 삭제
17   | 함께         | 27,988          | 0.5847     | 삭제
18   | 가능         | 27,